In [21]:

# Run the following code, update the path and you should have a CSV and EXCEL file with the current data available 
# from NGFS Short Term Physical Risk.


# NGFS short-term (Phase 5) — FULL dump of hazards × sectors × regions for 4 families
# pip install pyam-iamc ixmp4 pandas xlsxwriter

from pyam import iiasa
import pandas as pd
from pathlib import Path

# ---------------------- CONFIG ----------------------
OUTDIR = Path.home() / "Documents" / "Summer_intern"
OUTDIR.mkdir(parents=True, exist_ok=True)
CSV_PATH  = OUTDIR / "NGFS_shortterm_FULL_timeseries_New.csv"
XLSX_PATH = OUTDIR / "NGFS_shortterm_FULL_timeseries_New.xlsx"  # Excel may be large

YEARS = list(range(2023, 2031))          # short-term window
FAMILIES = [
    "capital_destruction",
    "production_lost",
    "productivity_loss",
    "labour_productivity_loss",
]

# Filters (use "*" to include all available)
MODELS    = "*"       # e.g., ["GEM-E3","CLIMACRED"]
SCENARIOS = "*"       # e.g., ["HWTP","SWUC","DAPS","DIRE"]
REGIONS   = "*"       # e.g., ["World","EU-15","China","USA"]

SAVE_CSV  = True
SAVE_XLSX = True

# ------------------ CONNECT -------------------------
con = iiasa.Connection()
db_name = None
for name in con.valid_connections:
    low = name.lower()
    if "ngfs" in low and ("short" in low or "short_term" in low):
        db_name = name
        break
if db_name is None:
    raise RuntimeError("NGFS short-term database not found on IIASA.")
con.connect(db_name)
print(f"Connected to IIASA DB: {db_name}")

# ------------------ HELPERS -------------------------
def split_var(v: str):
    parts = v.split("|")
    family = parts[0] if len(parts) > 0 else ""
    hazard = parts[1] if len(parts) > 1 else ""
    sector = "|".join(parts[2:]) if len(parts) > 2 else ""
    return family, hazard, sector if sector else "(total)"

def add_cols(dfp: pd.DataFrame) -> pd.DataFrame:
    fam_haz_sec = dfp["variable"].apply(split_var)
    dfp["family"] = fam_haz_sec.map(lambda x: x[0])
    dfp["hazard"] = fam_haz_sec.map(lambda x: x[1])
    dfp["sector"] = fam_haz_sec.map(lambda x: x[2])
    return dfp

# ------------------ DISCOVER VARIABLES ---------------
all_vars = con.variables()
family_vars = {}
for fam in FAMILIES:
    fam_low = fam.lower()
    fam_list = [v for v in all_vars if v.lower().startswith(fam_low + "|")]
    if fam_list:
        family_vars[fam] = sorted(fam_list)
    else:
        print(f"[WARN] No variables found for family: {fam}")

if not family_vars:
    raise RuntimeError("No variables discovered for the requested families.")

# ------------------ QUERY (FAMILY-BY-FAMILY) --------
dfs = []
for fam, var_list in family_vars.items():
    print(f"Querying family: {fam} ({len(var_list)} variables)")
    idf = con.query(
        model=MODELS, scenario=SCENARIOS, region=REGIONS,
        variable=var_list, default_only=True
    )
    if idf.empty:
        print(f"[WARN] Empty result for family: {fam}")
        continue
    idf = idf.filter(year=YEARS)
    pdf = idf.as_pandas()
    if pdf.empty:
        print(f"[WARN] No data in {YEARS[0]}–{YEARS[-1]} for {fam}")
        continue
    pdf = add_cols(pdf)
    dfs.append(pdf)

if not dfs:
    raise RuntimeError("No data returned for any family; relax filters or check availability.")

full = pd.concat(dfs, ignore_index=True)

# Order & clean columns
cols = ["family","hazard","sector","model","scenario","region","unit","year","value","variable"]
for c in cols:
    if c not in full.columns:
        full[c] = None
full = full[cols].sort_values(["family","hazard","sector","region","model","scenario","year"]).reset_index(drop=True)

# ------------------ SAVE ----------------------------
if SAVE_CSV:
    full.to_csv(CSV_PATH, index=False)
    print(f"CSV saved -> {CSV_PATH}")

if SAVE_XLSX:
    # Compatible Excel writer (no unsupported 'options' kwarg)
    with pd.ExcelWriter(XLSX_PATH, engine="xlsxwriter") as xlw:
        # Disable URL auto-detection if available (prevents long strings becoming hyperlinks)
        try:
            xlw.book.strings_to_urls = False
        except Exception:
            pass

        # One sheet per family (Excel sheet name ≤ 31 chars)
        for fam in FAMILIES:
            sub = full[full["family"] == fam]
            if sub.empty:
                continue
            sheet_name = fam[:31]
            sub.to_excel(xlw, sheet_name=sheet_name, index=False)
    print(f"Excel saved -> {XLSX_PATH}")

print("Done.")


Connected to IIASA DB: ngfs_phase_5_short_term
Querying family: capital_destruction (450 variables)
Querying family: production_lost (450 variables)
Querying family: productivity_loss (400 variables)
Querying family: labour_productivity_loss (50 variables)
CSV saved -> /Users/noenotter/Documents/Summer_intern/NGFS_shortterm_FULL_timeseries_Try.csv
Excel saved -> /Users/noenotter/Documents/Summer_intern/NGFS_shortterm_FULL_timeseries_Try.xlsx
Done.
